In [2]:
!pip install yt-dlp --quiet
!pip install moviepy --quiet
!pip install pytube moviepy opencv-python-headless --quiet


In [38]:
import os
import yt_dlp


video_url = "https://www.youtube.com/watch?v=l-gQLqv9f4o"
output_path = 'long_video.mp4'

ydl_opts = {
    'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
    'outtmpl': output_path,
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([video_url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=l-gQLqv9f4o
[youtube] l-gQLqv9f4o: Downloading webpage
[youtube] l-gQLqv9f4o: Downloading tv client config
[youtube] l-gQLqv9f4o: Downloading tv player API JSON
[youtube] l-gQLqv9f4o: Downloading ios player API JSON


[youtube] l-gQLqv9f4o: Downloading m3u8 information
[info] l-gQLqv9f4o: Downloading 1 format(s): 18
[download] long_video.mp4 has already been downloaded
[download] 100% of   20.18MiB


In [ ]:
# --- Step 3: Slice video into 2-minute chunks
from moviepy.editor import VideoFileClip

video = VideoFileClip("long_video.mp4")
chunk_duration = 2 * 45  # seconds
chunks = []

for start in range(0, int(video.duration), chunk_duration):
    end = min(start + chunk_duration, video.duration)
    clip = video.subclip(start, end)
    filename = f"chunk_{start}_{end}.mp4"
    clip.write_videofile(filename, codec='libx264', audio=False)
    chunks.append((filename, start, end))





t:   5%|▌         | 261/4982 [01:39<08:55,  8.82it/s, now=None]




t:   5%|▌         | 261/4982 [01:39<08:55,  8.82it/s, now=None]

t:   8%|▊         | 337/4316 [00:54<14:04,  4.71it/s, now=None]

Moviepy - Building video chunk_0_90.mp4.
Moviepy - Writing video chunk_0_90.mp4






t:   0%|          | 0/2158 [00:00<?, ?it/s, now=None]


t:   0%|          | 5/2158 [00:00<00:44, 48.25it/s, now=None]


t:   0%|          | 10/2158 [00:00<01:06, 32.32it/s, now=None]


t:   1%|          | 14/2158 [00:00<01:04, 33.29it/s, now=None]


t:   1%|          | 18/2158 [00:00<01:02, 34.31it/s, now=None]


t:   1%|          | 22/2158 [00:00<01:08, 31.28it/s, now=None]


t:   1%|          | 26/2158 [00:00<01:09, 30.83it/s, now=None]


t:   1%|▏         | 31/2158 [00:00<01:01, 34.82it/s, now=None]


t:   2%|▏         | 35/2158 [00:01<01:00, 34.87it/s, now=None]


t:   2%|▏         | 39/2158 [00:01<01:06, 31.67it/s, now=None]


t:   2%|▏         | 43/2158 [00:01<01:10, 30.19it/s, now=None]


t:   2%|▏         | 47/2158 [00:01<01:12, 29.27it/s, now=None]


t:   2%|▏         | 50/2158 [00:01<01:11, 29.30it/s, now=None]


t:   3%|▎         | 54/2158 [00:01<01:10, 29.97it/s, now=None]


t:   3%|▎         | 58/2158 [00:01<01:07, 30.89it/s, now=None]


t:   3%|▎         | 62/2158 [00:

In [164]:
import cv2
import numpy as np

import cv2
import numpy as np

def extract_all_features(path):
    cap = cv2.VideoCapture(path)
    brightness_values = []
    contrast_values = []
    motion_values = []
    dominant_color_rgb = []
    prev_gray = None
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        brightness_values.append(np.mean(gray))
        contrast_values.append(np.std(gray))

        if prev_gray is not None:
            diff = cv2.absdiff(prev_gray, gray)
            motion = np.mean(diff)
            motion_values.append(motion)
        prev_gray = gray

        frame_count += 1

        mean_rgb = np.mean(frame, axis=(0, 1))  # (R, G, B)
        dominant_color_rgb.append(mean_rgb)


    cap.release()

    return {
        'total_frames_x': frame_count,
        'overall_avg_brightness': np.mean(brightness_values) if brightness_values else 0,
        'avg_motion_intensity': np.mean(motion_values) if motion_values else 0,
        'total_frames_y': frame_count,
        'avg_contrast': np.mean(contrast_values) if contrast_values else 0,
        'total_frames_with_faces': 0,
        'angry': 0, 'disgust': 0, 'fear': 0, 'happy': 0, 'sad': 0, 'surprise': 0, 'neutral': 0,
        'face_presence_ratio': 0, 'avg_shot_length_frames': 0, 'total_shots': 0,
        'dominant_color_r': 0, 'dominant_color_g': 0, 'dominant_color_b': 0,
        'views_per_second': 0, 'likes_to_views_ratio': 0, 'title_length': 0,
        'has_hashtags': 0, 'has_emoji': 0
    }



In [133]:
print("Feature columns used in engine:", list(X_clip.columns))  # in clipping engine


🎬 Feature columns used in engine: ['total_frames_x', 'overall_avg_brightness', 'avg_motion_intensity', 'total_frames_y', 'avg_contrast', 'total_frames_with_faces', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio', 'avg_shot_length_frames', 'total_shots', 'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b']


In [119]:
features = []
for filename, start, end in chunks:
    feats = extract_all_features(filename)
    feats['start'] = start
    feats['end'] = end
    features.append(feats)

import pandas as pd
feature_df = pd.DataFrame(features)

takes long2 load

In [165]:
import joblib
from google.colab import drive

drive.mount('/content/drive')

clf = joblib.load("/content/drive/MyDrive/clipfarm_random_forest.joblib")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [175]:
actual_feature_columns = ['total_frames_x', 'overall_avg_brightness', 'avg_motion_intensity',
    'total_frames_y', 'avg_contrast', 'total_frames_with_faces', 'angry',
    'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio',
    'avg_shot_length_frames', 'total_shots', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b',
    'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji'
]

X_clip = feature_df[feature_columns].copy()
X_clip = X_clip.drop(columns=['clean_title', 'label', 'title', 'video_file', 'video_id'], errors='ignore')


In [169]:
"""# Force all required columns to exist (some may be missing if not extracted)
for col in feature_columns:
    if col not in feature_df.columns:
        feature_df[col] = 0
        print(f"Added missing column: {col}")"""

In [176]:
"""X_clip = feature_df.loc[:, feature_columns]"""


In [189]:
# force correct columns (order + fill missing with 0)
X_clip = feature_df.reindex(columns=feature_columns, fill_value=0)

In [190]:
print("Available columns:", feature_df.columns.tolist())


Available columns: ['overall_avg_brightness', 'avg_contrast', 'avg_motion_intensity', 'total_frames_x', 'total_frames_y', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio', 'avg_shot_length_frames', 'total_shots', 'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji', 'video_id', 'label', 'title', 'clean_title', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b', 'video_file', 'total_frames_with_faces', 'start', 'end']


In [191]:
print("Columns expected by model:")
print(clf.feature_names_in_.tolist())

print("\nColumns provided to model:")
print(X_clip.columns.tolist())


🔍 Columns expected by model:
['total_frames_x', 'overall_avg_brightness', 'avg_motion_intensity', 'total_frames_y', 'avg_contrast', 'total_frames_with_faces', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio', 'avg_shot_length_frames', 'total_shots', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b', 'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji']

🧪 Columns provided to model:
['video_file', 'total_frames_x', 'overall_avg_brightness', 'avg_motion_intensity', 'total_frames_y', 'avg_contrast', 'total_frames_with_faces', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio', 'avg_shot_length_frames', 'total_shots', 'clean_title', 'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b', 'video_id', 'label', 'title']


In [192]:
# Optional: Sanity check
print("\nColumns going into classifier:\n", X_clip.columns.tolist())
print("\n Shape of X_clip:", X_clip.shape)



✅ Columns going into classifier:
 ['video_file', 'total_frames_x', 'overall_avg_brightness', 'avg_motion_intensity', 'total_frames_y', 'avg_contrast', 'total_frames_with_faces', 'angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral', 'face_presence_ratio', 'avg_shot_length_frames', 'total_shots', 'clean_title', 'views_per_second', 'likes_to_views_ratio', 'title_length', 'has_hashtags', 'has_emoji', 'dominant_color_r', 'dominant_color_g', 'dominant_color_b', 'video_id', 'label', 'title']

✅ Shape of X_clip: (2, 29)


In [194]:
# ensure strict column type and order match
X_clip.columns = X_clip.columns.astype(str)
X_clip = X_clip.loc[:, clf.feature_names_in_]


In [195]:
predictions = clf.predict(X_clip)

In [196]:
feature_df['viral'] = predictions

print("\n Potential viral clips:")
for _, row in feature_df[feature_df['viral'] == 1].iterrows():
    start = int(row['start'])
    end = int(row['end'])
    print(f"→ {start//3600:02}:{(start%3600)//60:02}:{start%60:02} - {end//3600:02}:{(end%3600)//60:02}:{end%60:02}")



🔍 Potential viral clips:
→ 00:00:00 - 00:02:00
→ 00:02:00 - 00:03:27


In [202]:
importances = clf.feature_importances_

# create a dictionary mapping feature names used by the model to their importances
model_feature_importance_dict = dict(zip(clf.feature_names_in_, importances))

print("\n Potential viral clips with reasons:")

# filter feature_columns to only include those the model was trained on
# this prevents trying to convert non-numeric columns like 'video_file' to float
feature_columns = [col for col in feature_columns if col in clf.feature_names_in_]


for _, row in feature_df[feature_df['viral'] == 1].iterrows():
    start = int(row['start'])
    end = int(row['end'])
    print(f"→ Viral Clip: {start//3600:02}:{(start%3600)//60:02}:{start%60:02} - {end//3600:02}:{(end%3600)//60:02}:{end%60:02}")

    # calculate top 5 contributing features using only numerical features
    # use the feature_importance_dict that aligns with the model's features
    feature_scores = {
        col: float(row[col]) * model_feature_importance_dict[col]
        for col in feature_columns # Use the filtered list
    }
    top_features = sorted(feature_scores.items(), key=lambda x: x[1], reverse=True)[:5]

    print("   Top contributing features:")
    for feat, score in top_features:
        # Access the original value from the row for printing
        print(f"   - {feat}: {row[feat]:.2f} (weighted score: {score:.4f})")


✨ Potential viral clips with reasons:
→ Viral Clip: 00:00:00 - 00:02:00
   📊 Top contributing features:
   - total_frames_x: 2878.00 (weighted score: 174.4242)
   - total_frames_y: 2878.00 (weighted score: 174.4242)
   - avg_contrast: 56.32 (weighted score: 4.2669)
   - overall_avg_brightness: 77.88 (weighted score: 1.1801)
   - avg_motion_intensity: 3.64 (weighted score: 0.2205)
→ Viral Clip: 00:02:00 - 00:03:27
   📊 Top contributing features:
   - total_frames_x: 2105.00 (weighted score: 127.5758)
   - total_frames_y: 2105.00 (weighted score: 127.5758)
   - avg_contrast: 62.94 (weighted score: 4.7682)
   - overall_avg_brightness: 74.99 (weighted score: 1.1363)
   - avg_motion_intensity: 3.68 (weighted score: 0.2228)


In [203]:
print(feature_df[['start', 'end', 'viral']])

   start     end  viral
0      0  120.00   True
1    120  207.77   True
